In [1]:
import numpy as np
import re
import math
import os
import time

### gensim

In [2]:
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

1、Dictionary

    将doc转成Dictionary字典 (id, freq)

In [3]:
documents = ["Human machine interface for lab abc computer applications".split(),
             "A survey of user opinion of computer system response time".split(),
             "The EPS user interface management system".split(),
             "System and human system engineering testing of EPS".split(),
             "Relation of user perceived response time to error measurement".split(),
             "The generation of random binary unordered trees".split(),
             "The intersection graph of paths in trees".split(),
             "Graph minors IV Widths of trees and well quasi ordering".split(),
             "Graph minors A survey".split()]

dictionary = Dictionary(documents)

In [4]:
len(dictionary),dictionary[0],type(dictionary)

(45, 'Human', gensim.corpora.dictionary.Dictionary)

    return_missing参数

In [7]:
new_doc=["this","is","bryant", "is", "big"]
#       return_missing : bool, optional
#             Also return missing tokens (that doesn't contains in current dictionary).
dictionary.doc2bow(new_doc, return_missing=True) 

([], {'big': 1, 'bryant': 1, 'is': 2, 'this': 1})

In [8]:
len(dictionary),dictionary[0],type(dictionary)

(45, 'Human', gensim.corpora.dictionary.Dictionary)

    allow_update参数

In [9]:
new_doc=["my","son", "is", "bryant", "and", "he", "is", "exelent"]
#         allow_update : bool, optional 
#             If True - update dictionary in the process (i.e. add new tokens and update frequencies).
dictionary.doc2bow(new_doc, allow_update=True)

[(20, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1)]

In [10]:
len(dictionary),dictionary[0],type(dictionary)   # 更新了6个token

(51, 'Human', gensim.corpora.dictionary.Dictionary)

    doc2idx 返回doc的id

In [11]:
new_doc=["my","son", "is", "bryant", "and", "he", "is", "bravo"]
dictionary.doc2idx(new_doc)   ### 不在词表中，返回-1

[49, 50, 48, 45, 20, 47, 48, -1]

    token2id : dict of (str, int) 字段
        token -> tokenId.

In [12]:
dictionary.token2id

{'A': 8,
 'EPS': 16,
 'Graph': 38,
 'Human': 0,
 'IV': 39,
 'Relation': 24,
 'System': 19,
 'The': 17,
 'Widths': 40,
 'abc': 1,
 'and': 20,
 'applications': 2,
 'binary': 29,
 'bryant': 45,
 'computer': 3,
 'engineering': 21,
 'error': 25,
 'exelent': 46,
 'for': 4,
 'generation': 30,
 'graph': 34,
 'he': 47,
 'human': 22,
 'in': 35,
 'interface': 5,
 'intersection': 36,
 'is': 48,
 'lab': 6,
 'machine': 7,
 'management': 18,
 'measurement': 26,
 'minors': 41,
 'my': 49,
 'of': 9,
 'opinion': 10,
 'ordering': 42,
 'paths': 37,
 'perceived': 27,
 'quasi': 43,
 'random': 31,
 'response': 11,
 'son': 50,
 'survey': 12,
 'system': 13,
 'testing': 23,
 'time': 14,
 'to': 28,
 'trees': 32,
 'unordered': 33,
 'user': 15,
 'well': 44}

    dfs字段  dict of (int, int)
        Document frequencies: token_id -> in how many documents contain this token.

In [13]:
dictionary.dfs

{0: 1,
 1: 1,
 2: 1,
 3: 2,
 4: 1,
 5: 2,
 6: 1,
 7: 1,
 8: 2,
 9: 6,
 10: 1,
 11: 2,
 12: 2,
 13: 3,
 14: 2,
 15: 3,
 16: 2,
 17: 3,
 18: 1,
 19: 1,
 20: 3,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 3,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 1,
 38: 2,
 39: 1,
 40: 1,
 41: 2,
 42: 1,
 43: 1,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1}

     num_docs : int   一共有多少文档
        Number of documents processed.

In [14]:
dictionary.num_docs

10

       merge_with： 合并两个Dictionary
          Merge another dictionary into this dictionary, mapping same tokens to the same ids and new tokens to new ids.


In [15]:
corpus_1, corpus_2 = [["a", "b", "c"]], [["a", "f", "f"]]
dct_1, dct_2 = Dictionary(corpus_1), Dictionary(corpus_2)
dct_1.token2id

{'a': 0, 'b': 1, 'c': 2}

In [16]:
transformer = dct_1.merge_with(dct_2)
dct_1.token2id

{'a': 0, 'b': 1, 'c': 2, 'f': 3}

    filter_extremes 接口
    
        no_below : int, optional   至少有几篇doc包含每个token。 len(all_docs) * 5% 吧
            Keep tokens which are contained in at least `no_below` documents.
            
        no_above : float, optional   其实就是len(doc_contain_token) / len(all_docs) <= no_above
            Keep tokens which are contained in no more than `no_above` documents  
            (fraction of total corpus size, not an absolute number).
            
        keep_n : int, optional  默认是10w
            Keep only the first `keep_n` most frequent tokens.

In [17]:
corpus = [["hello", "you", "are"], ["are", "big", "me"],['hello','she'],['hello','it']]
dct = Dictionary(corpus)
len(dct)

7

In [18]:
dct.filter_extremes(no_below=1, no_above=0.5, keep_n=100)
print(len(dct))
for i in dct:
    print(dct[i])

6
are
you
big
me
she
it


    cfs Document frequencies: token_id -> how many documents contain this token.

In [37]:
id_freq = sorted(dictionary.cfs.items(),key=lambda x:x[1], reverse=True)
# print(id_freq)
for i in id_freq:
    print(dictionary.id2token[i[0]] + ":\t\t\t", i[1])

of:			 7
user:			 3
system:			 3
The:			 3
and:			 3
trees:			 3
interface:			 2
computer:			 2
A:			 2
survey:			 2
response:			 2
time:			 2
EPS:			 2
Graph:			 2
minors:			 2
is:			 2
Human:			 1
machine:			 1
for:			 1
lab:			 1
abc:			 1
applications:			 1
opinion:			 1
management:			 1
System:			 1
human:			 1
engineering:			 1
testing:			 1
Relation:			 1
perceived:			 1
to:			 1
error:			 1
measurement:			 1
generation:			 1
random:			 1
binary:			 1
unordered:			 1
intersection:			 1
graph:			 1
paths:			 1
in:			 1
IV:			 1
Widths:			 1
well:			 1
quasi:			 1
ordering:			 1
my:			 1
son:			 1
bryant:			 1
he:			 1
exelent:			 1


2、corpus
        
        实际上就是把每篇文档结合字典Dictionary变成，doc_1 = [(id,freq),(id,freq)...]

In [38]:
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [39]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(3, 1),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1)],
 [(5, 1), (13, 1), (15, 1), (16, 1), (17, 1), (18, 1)],
 [(9, 1), (13, 1), (16, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)],
 [(9, 1),
  (11, 1),
  (14, 1),
  (15, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1)],
 [(9, 1), (17, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)],
 [(9, 1), (17, 1), (32, 1), (34, 1), (35, 1), (36, 1), (37, 1)],
 [(9, 1),
  (20, 1),
  (32, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1)],
 [(8, 1), (12, 1), (38, 1), (41, 1)]]

In [58]:
[[(dictionary.id2token[id], freq) for id, freq in cp] for cp in corpus[1:2]]

[[('computer', 1),
  ('A', 1),
  ('of', 2),
  ('opinion', 1),
  ('response', 1),
  ('survey', 1),
  ('system', 1),
  ('time', 1),
  ('user', 1)]]

In [59]:
# 问：如果doc中不存在某个word呢？

In [60]:
dictionary.filter_extremes(no_below=1, no_above=0.5, keep_n=10)

In [61]:
dictionary.token2id

{'A': 2,
 'The': 7,
 'and': 8,
 'computer': 0,
 'interface': 1,
 'response': 3,
 'survey': 4,
 'system': 5,
 'trees': 9,
 'user': 6}

In [62]:
corpus = [dictionary.doc2bow(doc) for doc in documents]  

In [63]:
corpus

[[(0, 1), (1, 1)],
 [(0, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(1, 1), (5, 1), (6, 1), (7, 1)],
 [(5, 1), (8, 1)],
 [(3, 1), (6, 1)],
 [(7, 1), (9, 1)],
 [(7, 1), (9, 1)],
 [(8, 1), (9, 1)],
 [(2, 1), (4, 1)]]

In [ ]:
# 此时不存在词表dictionary中的token是不会存在corpus中的。

3、训练接口

    LdaModel  Train and use Online Latent Dirichlet Allocation (OLDA) models  用在线变分贝叶斯推断法求解
    
        chunksize :  int, optional  2000
            Number of documents to be used in each training chunk.  每次训练时的doc数目，类似于batch_size,一次性给入2000篇文章，一次性给入越多，性能越好，该指标会略微影响最终结果
            
        passes : int, optional     1
            Number of passes through the corpus during training.   遍历整个corpus的迭代数，类似于epoch
        
        alpha : {numpy.ndarray, str}, optional    文档-主题DIR分布的超参
            Can be set to an 1D array of length equal to the number of expected topics that expresses
            our a-priori belief for the each topics' probability.
            Alternatively default prior selecting strategies can be employed by supplying a string:

                * 'asymmetric': Uses a fixed normalized asymmetric prior of `1.0 / topicno`.
                * 'auto': Learns an asymmetric prior from the corpus (not available if `distributed==True`).
    
        eta : {float, np.array, str}, optional    主题-词DIR分布的超参
            A-priori belief on word probability, this can be:

                * scalar for a symmetric prior over topic/word probability,
                * vector of length num_words to denote an asymmetric user defined probability for each word,
                * matrix of shape (num_topics, num_words) to assign a probability for each word-topic combination,
                * the string 'auto' to learn the asymmetric prior from the data.
        
        iterations : int, optional            做推断时，最大迭代次数
            Maximum number of iterations through the corpus when inferring the topic distribution of a corpus.
            
    LdaMulticore An optimized implementation of the LDA algorithm, able to harness the power of multicore CPUs. LdaModel的多核版本
    
    
Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is.


In my experience, topic coherence score, in particular, has been more helpful.   从经验上看，主题一致性的评价指标更有用

4、pyLDAvis

        So how to infer pyLDAvis’s output?   
        
        Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.
        左边的圈圈越大，表示主题内部越相关

        A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant. 好的模型结果应该是圈圈之间尽可能不重叠，且很大

        A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

        Alright, if you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.